In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import json
import tensorflowjs as tfjs

## Import Text

In [16]:
with open("./trainingData/sherlockS.txt", 'r', encoding='utf-8') as file:
    text = file.read()

## Tokenize Text

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [18]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print(input_sequences[:20])

[[268, 61], [268, 61, 19], [268, 61, 19, 93], [268, 61, 19, 93, 14], [268, 61, 19, 93, 14, 94], [268, 61, 19, 93, 14, 94, 1], [268, 61, 19, 93, 14, 94, 1, 95], [268, 61, 19, 93, 14, 94, 1, 95, 6], [18, 145], [18, 145, 96], [18, 145, 96, 62], [18, 145, 96, 62, 269], [18, 145, 96, 62, 269, 97], [18, 145, 96, 62, 269, 97, 146], [18, 145, 96, 62, 269, 97, 146, 147], [98, 148], [98, 148, 10], [98, 148, 10, 7], [98, 148, 10, 7, 63], [98, 148, 10, 7, 63, 93]]


In [19]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

print(input_sequences[:8])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0 268  61]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 268  61  19]
 [  0   0   0   0   0   0   0   0   0   0   0   0 268  61  19  93]
 [  0   0   0   0   0   0   0   0   0   0   0 268  61  19  93  14]
 [  0   0   0   0   0   0   0   0   0   0 268  61  19  93  14  94]
 [  0   0   0   0   0   0   0   0   0 268  61  19  93  14  94   1]
 [  0   0   0   0   0   0   0   0 268  61  19  93  14  94   1  95]
 [  0   0   0   0   0   0   0 268  61  19  93  14  94   1  95   6]]


In [20]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

print(X[8])
print(y[8])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 18]
145


In [21]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## Create Model Architecture

In [22]:
model = Sequential()
model.add(Embedding(total_words, 400, input_length=max_sequence_len-1))
model.add(LSTM(400))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 400)           372000    
                                                                 
 lstm_1 (LSTM)               (None, 400)               1281600   
                                                                 
 dense_1 (Dense)             (None, 930)               372930    
                                                                 
Total params: 2026530 (7.73 MB)
Trainable params: 2026530 (7.73 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Train Model

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
71/71 [==============================] - 8s 89ms/step - loss: 6.3055 - accuracy: 0.0311
Epoch 2/200
71/71 [==============================] - 6s 88ms/step - loss: 5.7743 - accuracy: 0.0529
Epoch 3/200
71/71 [==============================] - 6s 90ms/step - loss: 5.5546 - accuracy: 0.0613
Epoch 4/200
71/71 [==============================] - 6s 89ms/step - loss: 5.3208 - accuracy: 0.0760
Epoch 5/200
71/71 [==============================] - 6s 91ms/step - loss: 5.0102 - accuracy: 0.1098
Epoch 6/200
71/71 [==============================] - 6s 88ms/step - loss: 4.5736 - accuracy: 0.1502
Epoch 7/200
71/71 [==============================] - 6s 90ms/step - loss: 4.0538 - accuracy: 0.2058
Epoch 8/200
71/71 [==============================] - 6s 89ms/step - loss: 3.4879 - accuracy: 0.2796
Epoch 9/200
71/71 [==============================] - 6s 89ms/step - loss: 2.8937 - accuracy: 0.3729
Epoch 10/200
71/71 [==============================] - 6s 89ms/step - loss: 2.3261 - accuracy: 0.5080

## Save Model for JS

In [24]:

output_dir = "./tfjs_models/RNN1" 
tfjs.converters.save_keras_model(model, output_dir)

## Save word index

In [25]:
# wordIndex = {}
# for word in tokenizer.word_index:
#     wordIndex[word] = tokenizer.word_index[word]

# with open('./tokenizer/wordIndex.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(wordIndex, ensure_ascii=False))

## Save Max Sequence Length

In [26]:
# with open('./tokenizer/max_sequence_len.txt', 'w') as f:
#     f.write(str(max_sequence_len))